# DRS Data Services

At first, the server is empty.

In [2]:
import requests
base_url = 'http://localhost:8080/ga4gh/dos/v1'
token = \
    "Bearer ya29.Gly2BqYpWKJ5MSc-I6BHGAOadOofBNt5vreFkr9Kck5fMUa-TiNxWpZ2i_Fl1NCNBtujPoew8XDnyHeSG__ZzyJ7TIxpHBnjzduewaUgeiA8OFrpsDjVGXMxd_xoeQ"
headers = {"Authorization": token}

data_objects = requests.post('{}/dataobjects/list'.format(base_url), json={"page_token": ""}, headers=headers).json()
print(data_objects)

{u'data_objects': []}


Then interacting with a Data Management Service (or an app in dos_connect) we perform CreateDataObject requests.

In [ ]:

!GOOGLE_APPLICATION_CREDENTIALS=/home/david/Downloads/human-cell-atlas-travis-test-0a109931bb95.json python -m dos_connect.apps.inventory.gs_inventory \
    --dos_server http://localhost:8080 \
    --api_key "$token" \
    -kn "Authorization" \
    org-humancellatlas-dss-checkout-dev

Now the Data Objects are registered in DRS.

In [25]:
data_objects = requests.post(
    '{}/dataobjects/list'.format(base_url),
    json={"page_token": ""},
    headers=headers).json()
print(data_objects)

{u'data_objects': [{u'updated': u'2019-02-19T21:04:29.228000+00:00', u'created': u'2019-02-19T21:04:29.228000+00:00', u'current': True, u'version': u'2019-02-19T14:48:28.676713Z', u'urls': [{u'url': u'https://www.googleapis.com/download/storage/v1/b/org-humancellatlas-dss-checkout-dev/o/blobs%2F0e48fdc80be7d661268c38383f770cfdeaa9978d8670a9231ae9950669811b85.f718753375f4714e748175fc42d3bcd55cbc8fb7.91a9c99105605821f6d88205ff932a65.f8759c6f?generation=1550610269229174&alt=media', u'system_metadata': {u'contentType': u'application/octet-stream', u'event_type': u'ObjectCreated:Put', u'bucket': u'org-humancellatlas-dss-checkout-dev', u'crc32c': u'+HWcbw==', u'etag': u'CPagobLZyOACEAE=', u'location': u'US', u'storageClass': u'MULTI_REGIONAL'}}, {u'url': u'gs://org-humancellatlas-dss-checkout-dev/blobs/0e48fdc80be7d661268c38383f770cfdeaa9978d8670a9231ae9950669811b85.f718753375f4714e748175fc42d3bcd55cbc8fb7.91a9c99105605821f6d88205ff932a65.f8759c6f'}], u'checksums': [{u'checksum': u'CPagobLZy

We can access the file using its media URL in Google Cloud.

In [26]:
print(data_objects['data_objects'][0]['urls'][1]['url'])
my_url = data_objects['data_objects'][0]['urls'][1]['url']

gs://org-humancellatlas-dss-checkout-dev/blobs/0e48fdc80be7d661268c38383f770cfdeaa9978d8670a9231ae9950669811b85.f718753375f4714e748175fc42d3bcd55cbc8fb7.91a9c99105605821f6d88205ff932a65.f8759c6f


In [27]:
!gsutil cp $my_url myfile.tmp

Copying gs://org-humancellatlas-dss-checkout-dev/blobs/0e48fdc80be7d661268c38383f770cfdeaa9978d8670a9231ae9950669811b85.f718753375f4714e748175fc42d3bcd55cbc8fb7.91a9c99105605821f6d88205ff932a65.f8759c6f...
/ [1 files][  419.0 B/  419.0 B]                                                
Operation completed over 1 objects/419.0 B.                                      


In [28]:
!cat myfile.tmp

{
  "describedBy": "https://schema.humancellatlas.org/type/file/6.1.1/sequence_file",
  "schema_type": "file",
  "file_core": {
    "file_name": "R2.fastq.gz",
    "file_format": "fastq.gz"
  },
  "read_index": "read2",
  "lane_index": 1,
  "provenance": {
    "document_id": "78971d24-b317-4f5b-9c95-e606905414ab",
    "submission_date": "2018-07-22T23:38:36.122Z",
    "update_date": "2018-07-22T23:39:37.631Z"
  }
}


## Sign a Data Object


In [29]:
des_url = "http://localhost:8080/ga4gh/des/v1/sign"

signed_url = requests.get(
    "{}/{}".format(des_url, data_objects['data_objects'][0]['id']),
    headers=headers)
print(signed_url.text)

https://storage.googleapis.com/org-humancellatlas-dss-checkout-dev/blobs%2F0e48fdc80be7d661268c38383f770cfdeaa9978d8670a9231ae9950669811b85.f718753375f4714e748175fc42d3bcd55cbc8fb7.91a9c99105605821f6d88205ff932a65.f8759c6f?Expires=1550616619&GoogleAccessId=david4096%40human-cell-atlas-travis-test.iam.gserviceaccount.com&Signature=m4jUw9nw%2BEtIdsgw1A1qtk7FwpuGD8aLIZOygPmbdK8ZE%2F5OjeRWPD1Z2Xw29lzzhWis39%2FmNhY%2BbSAdN1Y5Fhxl5%2FGbVzzun6Yjhsr2Q5tpshp8KmdblpaGKzIDFNHDd9wd1HfY2aSoN2MvNsjEKT08Z4l5SpYMfAqOjRftviR2vj0XneCsXBNl8FtgKgvMr5AWsKmTsVtikTGznEInIEfaXZbbpPELTnXpCv%2BlrkUdbNET9PdAEBn2CAvtwkCiZ6ni9INXdfyXcxd0p0EQt%2BHtALz5N8APcBwwZHeZT5wjkxphMzJ5knsrNgtQeu2%2F3EAzna%2BrysJBWf%2FwrXCHZw%3D%3D
